this is folk of the1owls's "Ridge & Huber - 3 Pointer (M)"  with Japanese caption.  
https://www.kaggle.com/the1owl/ridge-huber-3-pointer-m  
just for my understanding

In [1]:
# ライブラリを読み込む
import pandas as pd
import numpy as np
from sklearn import *
import os, glob

C:\Users\oakiyama\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\oakiyama\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\oakiyama\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationW

In [2]:
# データを読み込む
datafiles = sorted(glob.glob('../Winput/**.csv')) # 全データのパスを取得
# 全データをpd.Dataframeとして読み込んでdictionaryにまとめる
datafiles = {os.path.basename(file)[:-4]: pd.read_csv(file, encoding='latin-1') for file in datafiles}

In [3]:
# 全データをリストアップ
for i, key in enumerate(datafiles):
    print(i, key)

0 WNCAATourneyCompactResults_PrelimData2018
1 WSampleSubmissionStage1
2 WTeams
3 WNCAATourneySeeds_SampleTourney2018
4 WNCAATourneyDetailedResults_PrelimData2018
5 WCities
6 WGameCities
7 WGameCities_PrelimData2018
8 WRegularSeasonCompactResults
9 WNCAATourneySlots
10 WRegularSeasonCompactResults_PrelimData2018
11 WTeamSpellings
12 WSeasons
13 WSampleSubmissionStage2_SampleTourney2018
14 WSeasons_SampleTourney2018
15 WNCAATourneyCompactResults
16 WCities_PrelimData2018
17 WRegularSeasonDetailedResults_PrelimData2018
18 WNCAATourneySeeds


In [5]:
"""
SecondaryTourneyというカラムを追加して 'NCAA"もしくは'Regular'の値を与える
SecondaryTourneyはNCAAトーナメント以外のトーナメントのことでNIT, CBI, CIT, V16の４つがある．
ここにNCAA, Regularを追加して試合がどの大会に属するものかを表すカラムとして扱う．
"""
datafiles['WNCAATourneyCompactResults']['SecondaryTourney'] = 'NCAA'
# datafiles['WNCAATourneyDetailedResults']['SecondaryTourney'] = 'NCAA'
datafiles['WRegularSeasonCompactResults']['SecondaryTourney'] = 'Regular'
# datafiles['WRegularSeasonDetailedResults']['SecondaryTourney'] = 'Regular'

In [9]:
games =datafiles['WNCAATourneyCompactResults']
print("games.shape", games.shape)
games.head()

games.shape (1260, 9)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney
0,1998,137,3104,94,3422,46,H,0,NCAA
1,1998,137,3112,75,3365,63,H,0,NCAA
2,1998,137,3163,93,3193,52,H,0,NCAA
3,1998,137,3198,59,3266,45,H,0,NCAA
4,1998,137,3203,74,3208,72,A,0,NCAA


In [11]:
def experience(df, year, team):
    count = df.loc[(
            ((df.WTeamID==team) | (df.LTeamID==team))
            & (df.Season<year) 
            & (df.Season>=year-3) 
#             & (
#                 (df.SecondaryTourney==6)
#                (df.SecondaryTourney==1)
#                | (df.SecondaryTourney==2) 
#                | (df.SecondaryTourney==3)| (df.SecondaryTourney==4)
#                | (df.SecondaryTourney==5)
#               )
        )].shape[0]
    return count

In [12]:
experience(games, 1989, 1462)

0

In [17]:
season = pd.DataFrame(sorted(games['Season'].unique())[3:]+[2018,], columns=['Season'])
# season['TeamID'] = 1102
team = games['WTeamID'].unique().tolist() + games['LTeamID'].unique().tolist()
team = sorted(list(set(team)))
# print(len(team))
team = pd.DataFrame(team, columns=['TeamID'])

In [18]:
Season = sorted(season['Season'].unique())
for i in range(len(Season)):
    team[Season[i]] = team['TeamID'].apply(lambda x: experience(games, Season[i], x, ))

In [19]:
team.head()

,TeamID,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,3103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
1,3104,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3106,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,2,2
3,3107,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,3,4,4
4,3108,1,2,2,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0


In [138]:
# team.loc[team.TeamID==1462]
team.loc[team.TeamID==1231]

,TeamID,Season,1988,1989,1990,1991,1992,1993,1994,1995,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
100,1231,2018,7,8,10,5,7,9,12,12,...,5,3,1,0,3,6,6,4,4,4


In [141]:
y = 1990
t=1231
games.loc[((games.WTeamID==t) | (games.LTeamID==t)) 
          & ((games.Season==y-3) | (games.Season==y-2) | (games.Season==y-1))
          & (games.SecondaryTourney==6)]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney
130,1987,136,1231,92,1193,58,3,0,6
160,1987,138,1231,107,1120,90,3,0,6
178,1987,144,1231,88,1181,82,3,0,6
184,1987,146,1231,77,1261,76,3,0,6
186,1987,152,1231,97,1424,93,3,0,6
188,1987,154,1231,74,1393,73,3,0,6
215,1988,137,1350,72,1231,69,3,0,6
271,1989,137,1231,99,1206,85,3,0,6
293,1989,139,1231,92,1431,69,3,0,6
301,1989,143,1371,78,1231,65,3,0,6


In [20]:
team.to_csv("../additional/Wexperience.csv", index=None)